Se instalan los paquetes necesarios para el proyecto.

In [ ]:
!pip install contractions
!pip install inflect
!pip install pandas-profiling==2.7.1

Del paquete nltk, se obtienen las stopwords, el diccionario de palabras y el diccionario de palabras sinónimas.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

Se importan las librerias a ser utilizadas

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score, classification_report

Se automatiza el proceso del procesamiento de datos y el modelado en el pipeline.

Para el procesamiento de datos, se utiliza TfidfVectorizer para el vectorizado de los datos con los parametros, quita las palabras que no están en formato ASCII y tambien las stopwords inglesas.

In [4]:
pipe = Pipeline(
  steps = [
    ("tfidf", TfidfVectorizer(strip_accents="ascii", stop_words="english")),
    ("clf", RandomForestClassifier(criterion='entropy', max_depth= 300, n_estimators=500))
  ]
)


Se obtienen los datos del csb correspondiente y se proprocesa los labels para que sean de tipo categórico.

In [5]:
data = pd.read_csv('clinical_trials_on_cancer_data_clasificacion.csv', sep=',', encoding = 'utf-8')
data_t = data
data_t["label"] = data_t["label"].apply(lambda row: int(row.replace("__label__","")))

Se dividen los datos en entrenamiento y prueba con un ratio de 2:8 respectivamente.

In [6]:
objetivo = data_t["label"]
variables = data_t["study_and_condition"]
x_train, x_test, y_train, y_test = train_test_split(variables, objetivo, test_size=0.2, random_state=0)

Se usa gridsearch para buscar los mejores parametros para el modelo. (y luego se le asigna el mejor modelo al pipeline de forma manual)

In [12]:
parameters = {
    "clf__criterion": ["gini", "entropy"],
    "clf__max_depth": [300, 1000],
    "clf__n_estimators": [100, 500]
}

"""
grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)
best_model = grid_search.fit(x_train, y_train)
rfc_best = best_model.best_estimator_
print("best params: ", best_model.best_params_)
"""
pipe.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words='english', strip_accents='ascii')),
                ('clf',
                 RandomForestClassifier(criterion='entropy', max_depth=300,
                                        n_estimators=500))])

Se exporta el modelo como joblib

In [13]:
from joblib import dump, load

dump(pipe, 'modelo.joblib')

['modelo.joblib']

In [14]:
y_pred_train = pipe.predict(x_train)
y_pred_test = pipe.predict(x_test)
print('Exactitud sobre entrenamiento: %.2f' % accuracy_score(y_train, y_pred_train))
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred_test))

Exactitud sobre entrenamiento: 1.00
Exactitud sobre test: 0.80


Se obtiene una exactitud sobre el conjunto de prueba de 0.80 y una exactitud de entrenamiento de 1.00.

In [ ]:
print(pipe.score(x_test, y_test))
print(classification_report(y_test, y_pred_test))